# Risk diffusion experiment comparison indicators

In [1]:
from matplotlib import pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
shapes = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', '+']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'darkgreen', 'chocolate']

## Read network

In [2]:
from experiment.functions import func_get_lab_graph, save_graph
from experiment.lab2._0_config_ import LabSpace, SoftwareConfigs1
import os

Graphs = dict()
Labels = dict()
Types = dict()

for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        modelDir = os.path.join(LabSpace, softwareName)
        labDir = os.path.join(modelDir, '.lab2')
        
        G, labelDict, _, typeDict = func_get_lab_graph(modelDir, labDir, softwareName)
        Graphs[softwareName] = G
        Labels[softwareName] = labelDict
        Types[softwareName] = typeDict
        print(f'{softwareName} read completed')
        

In [3]:
from experiment.algorithms.rank.het_gm_rank import gc_rank, ggc_rank, iFit_rank, ksgc_rank, gcd_rank, ggcd_rank
from experiment.algorithms.rank.element_rank import element_rank, pr_rank
import networkx as nx

DRIs = dict()
DCs = dict()
BCs = dict()
CCs = dict()
PGs = dict()
ERs = dict()
GCs = dict()
GCDs = dict()
GGCs = dict()
GGCDs = dict()
iFits = dict()
KSGCs = dict()

ReCal = False

for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName]
        labDir = os.path.join(LabSpace, softwareName, '.lab2')
        
        print(f'{softwareName} starts calculating comparison methods')
        
        DRIs[softwareName] = dict(G.nodes.data('DRI'))

        for _, data in G.nodes(data=True):
            unG = G.to_undirected()
            
            if ReCal or 'dc' not in data:
                dc = nx.degree_centrality(G)
                DCs[softwareName] = dc
                dcList = [(key, {'dc': val}) for key, val in dc.items()]
                G.add_nodes_from(dcList)
            else:
                DCs[softwareName] = dict(G.nodes.data('dc'))
            print('\tDegree centrality calculation completed')
       
            if ReCal or 'cc' not in data:
                cc = nx.closeness_centrality(G)
                CCs[softwareName] = cc
                ccList = [(key, {'cc': val}) for key, val in cc.items()]
                G.add_nodes_from(ccList)
            else:
                CCs[softwareName] = dict(G.nodes.data('cc'))
            print('\tClose centrality calculation completed')

            if ReCal or 'bc' not in data:
                bc = nx.betweenness_centrality(G)
                BCs[softwareName] = bc
                bcList = [(key, {'bc': val}) for key, val in bc.items()]
                G.add_nodes_from(bcList)
            else:
                BCs[softwareName] = dict(G.nodes.data('bc'))
            print('\tMedian centrality calculation completed')
        
            if ReCal or 'pg' not in data:
                pg = nx.pagerank(G)
                PGs[softwareName] = pg
                pgList = [(key, {'pg': val}) for key, val in pg.items()]
                G.add_nodes_from(pgList)
            else:
                PGs[softwareName] = dict(G.nodes.data('pg'))
            print('\tPage rank calculation completed')
        
            if ReCal or 'er' not in data:
                er = element_rank(G)
                ERs[softwareName] = er
                erList = [(key, {'er': val}) for key, val in er.items()]
                G.add_nodes_from(erList)
            else:
                ERs[softwareName] = dict(G.nodes.data('er'))
            print('\tElement rank calculation completed')
            
            if ReCal or 'gc' not in data:
                gc = gc_rank(G)
                GCs[softwareName] = gc
                gcList = [(key, {'gc': val}) for key, val in gc.items()]
                G.add_nodes_from(gcList)
            else:
                GCs[softwareName] = dict(G.nodes.data('gc'))
            print('\tThe calculation of the center of gravity has been completed')
            
            if ReCal or 'gcd' not in data:
                gcd = gcd_rank(G)
                GCDs[softwareName] = gcd
                gcdList = [(key, {'gcd': val}) for key, val in gcd.items()]
                G.add_nodes_from(gcdList)
            else:
                GCDs[softwareName] = dict(G.nodes.data('gcd'))
            print('\tThe calculation of the center of gravity has been completed')
            
            if ReCal or 'iFit' not in data:
                pr = pr_rank(G)
                iFit = iFit_rank(G, pr)
                iFits[softwareName] = iFit
                iFitList = [(key, {'iFit': val}) for key, val in iFit.items()]
                G.add_nodes_from(iFitList)
            else:
                iFits[softwareName] = dict(G.nodes.data('iFit'))
            print('\tIFit calculation completed')
            
            if ReCal or 'ggc' not in data:
                ggc = ggc_rank(G)
                GGCs[softwareName] = ggc
                ggcList = [(key, {'ggc': val}) for key, val in ggc.items()]
                G.add_nodes_from(ggcList)
            else:
                GGCs[softwareName] = dict(G.nodes.data('ggc'))
            print('\tGGC calculation completed')
            
            if ReCal or 'ggcd' not in data:
                ggcd = ggcd_rank(G)
                GGCDs[softwareName] = ggcd
                ggcdList = [(key, {'ggcd': val}) for key, val in ggcd.items()]
                G.add_nodes_from(ggcdList)
            else:
                GGCDs[softwareName] = dict(G.nodes.data('ggcd'))
            print('\tGGC calculation completed')
            
            # if 'ksgc' not in data:
            #     ksgc = ksgc_rank(G)
            #     KSGCs[softwareName] = ksgc
            #     ksgcList = [(key, {'ksgc': val}) for key, val in ksgc.items()]
            #     G.add_nodes_from(ksgcList)
            # else:
            #     KSGCs[softwareName] = dict(G.nodes.data('ksgc'))
            
            break

        save_graph(labDir, G)

## The correlation between DRI and comparative methods

In [4]:
import pandas as pd
from experiment.functions import func_KendallTauTest

cols = ['DC', 'BC', 'CC', 'PageRank', 'ElementRank', 'GC', 'GCd', 'GGC', 'GGCd', 'iFit']
tauDf = pd.DataFrame(columns=['name', 'RCI', 'DC', 'BC', 'CC', 'PageRank', 'ElementRank', 'GC', 'GCd', 'GGC', 'GGCd', 'iFit'])

idx = 0
for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName]
        rci = DRIs[softwareName]
        
        compares = [
            ('DRI', DRIs[softwareName]),
            ('DC', DCs[softwareName]), 
            ('BC', BCs[softwareName]), 
            ('CC', CCs[softwareName]),
            ('PageRank', PGs[softwareName]),
            ('ElementRank', ERs[softwareName]),
            ('GC', GCs[softwareName]),
            ('GCd', GCDs[softwareName]),
            ('GGC', GGCs[softwareName]),
            ('GGCd', GGCDs[softwareName]),
            ('iFit', iFits[softwareName]),
            # ('VoteRank', VTs[softwareName]),
        ]
        
        tausAll, _ = func_KendallTauTest(rci, compares)
        taus, _ = func_KendallTauTest(rci, compares)
        taus['name'] = softwareName
        tauDf.loc[idx] = taus
        
        idx += 1
tauDf.set_index('name', inplace=True)        

directory = os.path.join(LabSpace, '.rdi')
if not os.path.isdir(directory):
    os.makedirs(directory)
    
tauDf.to_csv(os.path.join(directory, 'Kendall-tau.csv'))
tauDf[cols]

## Scatter plot of the top 30% nodes ranked by DRI and comparison algorithms

In [5]:
import shutil

saveDir = os.path.join(LabSpace, 'graph', 'DRI-kendell-tau-b')
if os.path.isdir(saveDir):
    shutil.rmtree(saveDir)
os.makedirs(saveDir)    

In [6]:
from experiment.functions import func_SortDictByValue
from matplotlib import pyplot as plt
import numpy as np
from experiment.functions import func_std

labs = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

fig, axs = plt.subplots(3, 3, figsize=(24, 18))
plt.subplots_adjust(hspace=0.28, wspace=0.22)

idx = 0
for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName]
        N = len(DRIs[softwareName])
        count = int(N * 0.3)
        
        rci = DRIs[softwareName]
        
        X = np.linspace(0, count-1, count)
        keys = list(func_SortDictByValue(DRIs[softwareName]).keys())[:count]
        dris = func_std([DRIs[softwareName][n] for n in keys])
        dcs = func_std([DCs[softwareName][n] for n in keys])
        bcs = func_std([BCs[softwareName][n] for n in keys])
        ccs = func_std([CCs[softwareName][n] for n in keys])
        pgs = func_std([PGs[softwareName][n] for n in keys])
        ers = func_std([ERs[softwareName][n] for n in keys])
        gcs = func_std([GCs[softwareName][n] for n in keys])
        gcds = func_std([GCDs[softwareName][n] for n in keys])
        ggcs = func_std([GGCs[softwareName][n] for n in keys])
        ggcds = func_std([GGCDs[softwareName][n] for n in keys])
        ifits = func_std([iFits[softwareName][n] for n in keys])
        
        ax = axs[int(idx / 3), idx % 3]
        ax.scatter(X, dcs, label='DC', marker=shapes[1], color=colors[1])
        ax.scatter(X, bcs, label='BC', marker=shapes[2], color=colors[2])
        # ax.scatter(X, ccs, label='CC', marker=shapes[3], color=colors[3])
        # ax.scatter(X, pgs, label='PageRank', marker=shapes[4], color=colors[4])
        # ax.scatter(X, ers, label='ElementRank', marker=shapes[8], color=colors[8])
        ax.scatter(X, gcs, label='GC', marker=shapes[3], color=colors[3])
        ax.scatter(X, gcds, label='$GC_d$', marker=shapes[4], color=colors[4])
        ax.scatter(X, ggcs, label='GGC', marker=shapes[5], color=colors[5])
        ax.scatter(X, ggcds, label='$GGC_d$', marker=shapes[6], color=colors[6])
        ax.scatter(X, ifits, label='iFIt', marker=shapes[7], color=colors[7])
        ax.scatter(X, dris, label='DRI', marker=shapes[0], color=colors[0])
        
        ax.set_xlabel('Function', fontdict={'size': 12})
        ax.set_ylabel('Score', fontdict={'size': 12})
        ax.set_title(f'{labs[idx]}) {softwareName}', y=-0.21, fontdict={'family': 'Times New Roman', 'size': 18})
        
        ax.legend()
        
        df = pd.DataFrame()
        df['id'] = X
        df['node-key'] = keys
        df['DRI'] = dris
        df['DC'] = dcs
        df['BC'] = bcs
        df['GC'] = gcs
        df['GC_d'] = gcds
        df['GGC'] = ggcs
        df['GGC_d'] = ggcds
        df['iFIt'] = ifits
        df.to_csv(os.path.join(saveDir, f'{softwareName}.csv'))
        
        idx += 1
        
plt.show()